In [150]:
# Import necessary libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [93]:
#Red flags según artículo
red_flags_celular = ["detalle", "detalles", "bloqueado", "roto", "rota", "dañado", "dañada", "malo", "mala"] #, "repuesto", "repuestos"]
red_flags_gpu = ["detalle", "detalles", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "deshuase", "no da imagen", "no da video"]
# Notebook [detalle, detalles, roto, rota, dañado, dañada, malo, mala, repuesto, repuestos, deshuase]

red_flags = red_flags_gpu

close_locations = ["Valparaiso", "Viña del Mar"]

In [128]:
# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 40000
max_price = 500000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
make = "tarjeta de video nvidia "
search_model = "rtx 3060"
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [96]:
# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Configurar las opciones de Chrome
options = Options()
# options.add_argument("--headless")  # Para ejecutar Chrome en modo sin cabeza (sin ventana)
options.add_experimental_option("detach", True) # Para que el driver no se cierre al terminar el script

# Inicializar el driver de Chrome
driver = webdriver.Chrome(service=service, options=options)
# Set up base url
base_url = "https://www.facebook.com/marketplace/"

#Set up full url
url = f"{base_url}{default_location}/search?availability={availability}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_since_listed}&query={make}{search_model}&exact={exact}"

In [97]:
# Visitar el sitio web
driver.get(url)

# Esperar un tiempo breve para que la página se cargue completamente
time.sleep(1) # 1 segundo

# Buscar el elemento de la X para ignorar el inicio de sesión
x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")

# Hacer clic en la X para ignorar el inicio de sesión
x_button.click()

# CONFIGURAR EL RADIO DE DISTANCIA

In [155]:
change_location_selector = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]')

# Obtener ubicación actual
current_location = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]/div[1]/div/span').text
print(f"La búsqueda se está realizando en: {current_location}")

# Hacer clic en el selector de ubicación
change_location_selector.click()

# Cambiar la ubicación de búsqueda
location = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/div/div/label/div/div[2]/input')

ActionChains(driver).click(location).perform()

# esperar 1 segundo
time.sleep(1)

La búsqueda se está realizando en: Curacaví, Region Metropolitana, Chile
  · En un radio de 60 kilómetros


In [153]:
location.clear()

In [156]:
location.send_keys(search_location)

location.send_keys(Keys.RETURN)

# Hay que hacer que seleccione una de las opciones del menu desplegable

In [158]:
# buscar selector de radio
radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label')

# Hacer clic en el selector de radio
radius_selector.click()

# Botones de seleccion de radio

elementos = driver.find_elements(By.XPATH, "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z']")

# Selecciona el radio en el menú

for elemento in elementos:
    numero = re.match(r"\d+", elemento.text).group()
    
    if int(numero) == radius:
        elemento.click()
        break

# Buscar el botón de aplicar
apply_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div')

# Hacer clic en el botón de aplicar
apply_button.click()

In [13]:
# Desplazarse hacia abajo para cargar más resultados
scroll_count = 6  # Número de veces a desplazar la página
scroll_delay = 2  # Retraso entre cada desplazamiento en segundos

for _ in range(scroll_count):
    # Ejecutar JavaScript para desplazarse hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar un momento para permitir que se cargue el contenido
    time.sleep(scroll_delay)

In [14]:
# Obtener el HTML de la página
html = driver.page_source

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')

In [15]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1j85h84")
locations_list = [location.text.strip() for location in locations_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [16]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
    # Agregar el precio limpio a la lista
    clean_prices_list.append(precio_limpio)

In [17]:
def get_description(url):
    driver.get(url)
    #time.sleep(1)
    
    try:
        # Buscar el elemento de la X para ignorar el inicio de sesión
        x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")
        # Hacer clic en la X para ignorar el inicio de sesión
        x_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        #time.sleep(1)
    except:
        pass

    try:

        ver_mas_button = driver.find_element(By.XPATH, "//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1lku1pv x1a2a7pz')]//*[contains(text(), 'Ver más')]")
        # Hacer clic en el botón "Ver más"
        ver_mas_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente

    except:
        pass
    
    html = driver.page_source
    item_soup = soup(html, 'html.parser')

    #description_spans = item_soup.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u")
    description_spans = item_soup.find('div', class_="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a")

    #Si no se encuentra la descripción, se le asigna "No description"
    if description_spans == None:
        description = "No description"
    else:
        description = description_spans.text.strip()

    words = description.split()  # Dividir la descripción en palabras
    if len(words) >= 2 and words[-2:] == ["Ver", "menos"]:
        # Si las últimas dos palabras son "Ver menos", eliminarlas
        description = " ".join(words[:-2]).strip()

    #print(description)
    return description


La parte de filtrado de resultados obtenidos es la más demorada de todas

In [18]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(search_model)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    # Check if the title contains the search model
    if re.search(patron, item):

        for flag in red_flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), item.lower()):
                #if flag == 'detalles' and 'sin detalles' in item.lower():
                if flag == 'detalles' and ('sin detalles' in item.lower() or 'no tiene detalles' in item.lower()):

                    # Si encuentra "detalles" y "sin detalles" está presente en el titulo del anuncio, entonces no es una bandera roja
                    continue
                else:
                    print(f"Se encontró una red flag en el siguiente título: {item}")
                    break
        else:
            items_dict["Titel"] = item
            items_dict["Model"] = search_model
            items_dict["Location"] = locations_list[i]
            items_dict["Price"] = clean_prices_list[i]
            items_dict["URL"] = urls_list[i]        

            # Add description to the items_list
            items_dict["Description"] = get_description(urls_list[i])

            # Check for red flags in the description
            description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas
            for flag in red_flags:
                if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                    #if flag == 'detalles' and 'sin detalles' in description:
                    if flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles' in description):
                        # Si encuentra "detalles" y "sin detalles" está presente en la descripción, entonces no es una bandera roja
                        continue
                    elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                        continue
                    else:
                        print(f"Se encontró un red flag en la siguiente descripción: {items_dict['Description']}")
                        break
                    
            else:  
                items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")


Se encontró un red flag en la siguiente descripción: Vendo o permuto mi gpu por alguna de mayor calidad puedo dar dinero a favor, el unico detalle de la gpu es que no tengo la caja

Se obtuvieron 18 resultados utilizables
De una búsqueda de 169 resultados


In [ ]:
# Cerrar la sesión del navegador
#driver.quit()

In [19]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Model,Location,Price,URL,Description
13,Tarjeta Grafica RTX 3060 12GB EDICION MSI TRIO GAMING X,rtx 3060,"Pichilemu, LI",310000,https://web.facebook.com/marketplace/item/351558407215466/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Vendo rtx 3060 uso aprox 1 año, esta impeque, nunca a dado problemas, la vendo por renovación. Precio conversable."
14,Con envio RTX 3060 NUEVA GDDR6,rtx 3060,"Concepción, BI",299990,https://web.facebook.com/marketplace/item/1472689960261095/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,Tarjeta grafica RTX 3060 MSI VENTUS X2\n8GB GDDR6\nNUEVA SIN USO
15,Tarjeta gráfica RTX 3060 ti,rtx 3060,"Concepción, BI",380000,https://web.facebook.com/marketplace/item/2202351740105952/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Si está publicado esta disponible Vendo tarjeta gráfica rtx 3060 ti 8gb muy superior a la 3060 normal. Muy poco uso universitario más que nada, por eso la vendo, está nueva aún con sus sellos de garantía en los tornillos y su cajita, precio conversable y también hago permuta más dinero"
16,Rtx 3060 ti Zotac,rtx 3060,"Coronel, BI",320000,https://web.facebook.com/marketplace/item/1117387296130626/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,Usada para gaming funcionando impecable full bundle
17,Asus Dual GeForce RTX 3060 Ti GDDR6X,rtx 3060,"Talca, ML",300000,https://web.facebook.com/marketplace/item/979222853774734/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Tarjeta gráfica ASUS Dual GeForce RTX 3060 Ti OC Edition 8GB GDDR6X con 1 MES de uso esta prácticamente nueva , FULL BUNDLE."


In [20]:
items_df

,Titel,Model,Location,Price,URL,Description
0,Tarjeta RTX 3060 / 12GB,rtx 3060,"Santiago, RM",240000,https://web.facebook.com/marketplace/item/707402961588896/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Tarjeta Grafica RTX 3060 Palit - 12 GB En buenas condiciones, no posee caja, puedo entregar en mi domicilio o puedo ir a domicilio. puedo hacer envíos a regiones."
1,RTX 3060 12gb,rtx 3060,"Viña del Mar, VS",300000,https://web.facebook.com/marketplace/item/712530737733192/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Vendo gpu rtx 3060 la compre para armar otro pc, pero nunca lo terminé, conversable"
2,RTX 3060 TI,rtx 3060,"Peñalolén, RM",280000,https://web.facebook.com/marketplace/item/385383284447373/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,Cuenta con todo sus accesorios ... Solo de uso de juegos como valorant ... Solo mandar pv ... Si realmente está interesado ... Gracias hasta 280.000 lo puedo dejar
3,Tarjeta de Video RTX 3060 Gigabyte OC 12GB,rtx 3060,"Santiago, RM",330000,https://web.facebook.com/marketplace/item/1593974338072268/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,Tarjeta de Video Open Box\nSin Uso
4,RTX 3060 TI,rtx 3060,"Santiago, RM",330000,https://web.facebook.com/marketplace/item/1112858196596615/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Vendo Tarjeta Gráfica Zotac RTX 3060 Ti por renovación: Pongo a la venta una gráfica Zotac RTX 3060 Ti, la cual ofrece un rendimiento excepcional en juegos y tareas de edición. Especificaciones: Marca: Zotac Modelo: RTX 3060 Ti Memoria: 8GB GDDR6 Interfaz: PCI Express 4.0 Salidas de video: 3x DisplayPort 1.4a, 1x HDMI 2.1 Núcleos CUDA: 4864 Reloj Base: 1410 MHz Reloj Boost: 1665 MHz Soporte para Ray Tracing: Sí Tecnologías de refrigeración: IceStorm 2.0, FREEZE Fan Stop Dimensiones: 231.9mm x 141.3mm x 41.5mm"
5,Tarjeta grafica Palit Geforce RTX 3060 Ti Dual 8GB,rtx 3060,"Santiago, RM",460000,https://web.facebook.com/marketplace/item/1141588373522877/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Tarjeta grafica, completamente nueva, se compró para un proyecto que al final nunca se realizó."
6,RTX 3060 12gb MSI GAMING X,rtx 3060,"Ñuñoa, RM",350000,https://web.facebook.com/marketplace/item/742229044707567/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"Tiene 4 meses de uso. Tengo su caja y los manuales. Está muy bien cuidada , jamás overclock ni nada raro . Vendo únicamente porque me cambio de tarjeta de video . Entrego probada ."
7,"VENDO POR UPGRADE MI RTX 3060 VISION, TIENE UN AÑO DE USO CON SUS SELLOS, TEMPERATURA PERFECTA.",rtx 3060,"Santiago, RM",280000,https://web.facebook.com/marketplace/item/374875895435295/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"RTX 3060 VISION 12GB, CON SU CAJA"
8,Zotac GeForce RTX 3060 Ti Twin Edge,rtx 3060,"Viña del Mar, VS",300000,https://web.facebook.com/marketplace/item/1064178344685643/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,FabricanteZotacGPUNVIDIA GeForce RTX 3060 TiMemoria8 GB GDDR6 (256 bit)BusPCI Express 4.0 x16Frecuencias core (base / boost / OC)1410 / 1665 / 1695 MHzFrecuencia memorias1750 MHz Especificaciones secundarias NúcleoNVIDIA Ampere GA104PerfilNormalRefrigeraciónVentiladorSlotsDual slotLargo222 mm.IluminaciónSimple¿Backplate?SíConectores de poder 1x 8 pines Puertos de video 3x DisplayPort 1x HDMI PRECIO Conversable
9,RTX 3060 TI MSI VENTUS X3,rtx 3060,"Santiago, RM",259000,https://web.facebook.com/marketplace/item/421509947027802/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD,"RTX 3060 TI Perfectas condiciones, puedo entregar a domicilio si es V region o enviar por mercadolibre."


In [21]:
# Extraer los precios y URLs de la lista de diccionarios
prices = [item["Price"] for item in items_list]
urls = [item["URL"] for item in items_list]

# Calcular el precio promedio
precio_promedio = np.mean(prices)

# Calcular el precio más barato
precio_mas_barato = np.min(prices)
indice_precio_mas_barato = np.argmin(prices)

# Calcular el percentil 10 más bajo
percentil_10_mas_bajo = np.percentile(prices, 10)

# Calcular la mediana de los precios
mediana_precios = np.median(prices)


# Imprimir los resultados
print("Precio promedio:", precio_promedio)
print("Mediana de precios:", mediana_precios)
print("Percentil 10 más bajo:", percentil_10_mas_bajo)
print("Precio más barato:", precio_mas_barato)
print("URL:", urls[indice_precio_mas_barato])

Precio promedio: 313277.22222222225
Mediana de precios: 300000.0
Percentil 10 más bajo: 253300.0
Precio más barato: 120000
URL: https://web.facebook.com/marketplace/item/989349442830460/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
